# **Modelo de predicción de la calificación de los profesores en el sitio web 'Los estudiantes'**

*Presentador por:*
---
**Juan Daniel Gomez Ríos.**
- Estudiante de Ciencias de la Computación de la Universidad Nacional de Colombia.
- Email: judgomezri@unal.edu.co
- github:

---

**Carlos Ernesto Isaza Carvajal.**
- Estudiante de Matemáticas de la Universidad Nacional de Colombia.
- Email: ceisazac@unal.edu.co
- github: carer70

---

**Tania Valentina Delgado Castillo.**
- Estudiante de Ciencias de la Computación de la Universidad Nacional de Colombia.
- Email: tvcastillod@unal.edu.co
- github: tvcastillod

---

**Jonnathan Lancheros Naranjo.**
- Estudiante de Matemáticas de la Universidad Nacional de Colombia.
- Email: jlancherosn@unal.edu.co
- github: jlancherosn

# Introducción

'Los estudiantes' es un sitio web (https://losestudiantes.co/) creado para evaluar de forma pública el desempeño de los profesores de algunas universidades de la ciudad de Bogotá, allí los estudiantes peden crear una cuenta desde la cuál pueden darle alguna nota numérica (un valor entre 1.5 y 5) al profesor con quien se haya visto algún curso y, además, dejar un comentario en el que se detalla la razón de la calificación en base a las fortalezas y las debilidades del profesor en determinado curso.

Como proyecto se tiene el objetivo de establecer una manera de predecir futuras calificaciones en base a los datos que al día de hoy se encuentran sobre algunos profesores del Departamento de Matemáticas de la Universidad Nacional de Colombia. Esto es, se quiere predecir la calificación numérica que obtendrá un profesor a partir de las palabras que contengan los comentarios, para esto no será necesario especificar a qué profesor estan asociados los datos de prueba sino que se agrupan en un mismo conjunto, básicamente solo nos fijamos en la variable numérica que representa la nota y en el string que origina cada comentario.

Particularmente, se ha visto este tipo de anáslisis de texto con dos ejemplos concretos, el primero de ellos basado en los comentarios de las películas que han visto algunos de los usuarios de Reddit, allí se quiere clasificar películas pero esta clasificación es binaria pues hay dos opciones: la película es buena o no lo es, el proceso de clasificación se hace por medio de regresión logística sobre datos etiquetados con 0 o 1. El otro ejemplo es sobre una serie de cuentos escritos por el argentino Hernan Casciari, en este caso se quiere hacer una exploración sobre los sentimientos positivos o negativos encontrados en el texto y esto se hace con ayuda de la librería *TextBlob* de Python. En ambos casos se hace necesario hacer scraping en los sitios web donde se almacenan los datos de interés, en nuestro caso también necesitamos recurrir al mismo proceso.

# Construcción de la base de datos

El scraping lo podemos hacer con ayuda de las librerías requests y  BeautifulSoup, la primera nos devuelve el código HTML de la página web desde el servidor donde se aloja y el segundo nos da la posibilidad de ir a lugares específicos de aquel código para extraer los datos concretos que necesitamos extraer.

In [0]:
#librerias que se utilizan
import numpy as np
import requests
import pandas as pd
from bs4 import BeautifulSoup
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
URL = 'https://losestudiantes.co/universidad-nacional/derecho/profesores/eulises-torres'
page = requests.get(URL)

print(page.content)

soup = BeautifulSoup(page.content, 'html.parser')

b'<!DOCTYPE html><html><head><meta charSet="utf-8" class="jsx-3623786364 next-head"/><meta name="viewport" content="width=device-width, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" class="jsx-3623786364 next-head"/><meta name="google-site-verification" content="OcQZjjFvF5i-Og82UscDc3G5CqpcbMyx3F5JsyDXtgM" class="jsx-3623786364 next-head"/><title class="jsx-3623786364 next-head">Eulises Torres | Los Estudiantes</title><meta name="description" content="Calificaciones de Eulises Torres. Profesor de Derecho en la Universidad Nacional. Bogot\xc3\xa1, Colombia. 2 Calificaciones: Muy buen profesor. Sus clases pueden tornarse un poco aburridas, pero sin duda se aprende. Es un amo..." class="jsx-3623786364 next-head"/><meta name="keywords" content="los estudiantes, Universidad de los Andes, uniandes, estudiantes, calificar profesores" class="jsx-3623786364 next-head"/><meta property="og:image" content="https://losestudiantes.co/static/images/logo.png" class="jsx-3623786364 next-head"

Cuando se selecciona la Universidad Nacional y el Pregrado Matemáticas en la página de inicio de Los estudiantes no es posible extraer desde allí las direcciones URL de cada profesor porque no aparecen listadas en el código HTML, entonces seleccionamos uno cualquiera (en nuestro caso el profesor Jose Luis Ramirez) y desde el código HTML de su página de calificaciones comenzamos a navegar y a construir el vector profesores. Se puede notar que al ejecutar el código el listado cambia a partir de la segunda entrada, esto se debe a la configuración propia del sitio web.

In [0]:
resultados = soup.find('div', class_ = 'jsx-3672521041')
opiniones = resultados.find_all('li', class_='jsx-571610088 post ')

vector_notas = []
vector_comentarios = []

for opinion in opiniones:
    nota = opinion.find('div', class_='jsx-571610088 leftCalificacion').find('span', class_ = 'jsx-571610088 numeroStats')
    comentario = opinion.find('div', class_ = 'jsx-571610088 calificacion').find('div', class_ = 'jsx-571610088 lineBreak')
    vector_notas.append(nota.text)
    vector_comentarios.append(comentario.text)

In [0]:
db = pd.DataFrame({'Nota': vector_notas, 'Comentario': vector_comentarios})
db.to_csv('losestudiantes.csv', index = False)
pd.read_csv('losestudiantes.csv')

,Nota,Comentario
0,4.5,Muy buen profesor. Sus clases pueden tornarse ...
1,5.0,"Es un gran docente, tiene gastos conocimientos..."


In [0]:
URL_ = 'https://losestudiantes.co/universidad-nacional/matematicas/profesores/'
page_ = requests.get(URL_)

print(page_.content)

soup = BeautifulSoup(page.content, 'html.parser') 

b'<!DOCTYPE html><html><head><meta charSet="utf-8" class="next-head"/><link rel="preload" href="/_next/static/AhvcrW_5yafFi9NiI8E8E/pages/_app.js" as="script"/><link rel="preload" href="/_next/static/AhvcrW_5yafFi9NiI8E8E/pages/_error.js" as="script"/><link rel="preload" href="/_next/static/runtime/webpack-1b21ac7d2170efbb4db5.js" as="script"/><link rel="preload" href="/_next/static/chunks/commons.084401de3ce27ff33141.js" as="script"/><link rel="preload" href="/_next/static/runtime/main-08cf410edc6b21720404.js" as="script"/><link rel="shortcut icon" href="/static/images/favicon.ico" type="image/x-icon"/><link rel="stylesheet" href="/static/css/bootstrap.min.css"/><link rel="stylesheet" href="/static/css/react-select.css"/><link rel="stylesheet" href="/static/css/font-awesome.min.css"/><link rel="stylesheet" href="/static/css/bootstrap-slider.min.css"/></head><body><div id="__next"><p data-reactroot="">El profesor o el link que buscas no existe</p></div><script>__NEXT_DATA__ = {"props":

In [0]:
profesores = ['jose-luis-ramirez']

for i in range (30):
  URL = 'https://losestudiantes.co/universidad-nacional/matematicas/profesores/' + profesores[i]
  page = requests.get(URL)
  soup = BeautifulSoup(page.content, 'html.parser')
  for link in soup.find_all('a'):
    link_ = str(link.get('href'))
    if ('/universidad-nacional/matematicas/profesores/' in link_ and link_[45:] not in profesores):
      profesores.append(link_[45:])
  #print(len(profesores))

print(len(profesores))
print(profesores)

115
['jose-luis-ramirez', 'margaret-johanna-garzon-merchan', 'herbert-alonso-dueñas-ruiz', 'omar-duque-gomez', 'lorenzo-maria-acosta-gempeler', 'oscar-javier-lopez-alfonso', 'jose-reinaldo-montanez-puentes', 'alexis-georges-denis-irlande', 'zulima-ortiz-bayona', 'freddy-rolando-hernandez-romero', 'hector-camilo-chaparro-gutierrez', 'john-alexander-cruz-morales', 'eduardo-cardenas-gomez', 'juan-carlos-galvis-arrieta', 'jorge-mauricio-ruiz-vera', 'jaime-andres-robayo-mesa', 'german-preciado-lopez', 'liliana-constanza-romero-marroquin', 'guillermo-rodriguez-blanco', 'juan-andres-montoya-arguello', 'edwin-aldemar-jiménez-quiroga', 'juan-carlos-juajibioy-otero', 'jose-jorge-sierra-molina', 'carolina-albarracin-hernandez', 'clara-marina-neira-uribe', 'efrain-camilo-pardo-garcia', 'felix-humberto-soriano-mendez', 'ibeth-marcela-rubio-perilla', 'armando-zarruk-rivera', 'daniel-nuñez-alarcón', 'ronald-gentil-rodriguez-giraldo', 'hernando-gaitan-orjuela', 'manuel-jair-medina-luna', 'ricardo-arie

Ahora, sobre cada componente del vector profesores buscamos las etiquetas donde estan ubicados la nota y el comentario, y extraemos su texto con ayuda de soup. A continuación se da un poco de contexto visual y el por qué de las etiquetas especificadas en las siguientes líneas de código.

![html](https://drive.google.com/uc?id=1WXTrJ0D_QXmyl4AJO-OncnIg2PlbQdBZ)

In [0]:
frames = []

for profesor in profesores:
  URL_ = 'https://losestudiantes.co/universidad-nacional/matematicas/profesores/'+profesor
  #print(profesor)
  page = requests.get(URL_)
  soup = BeautifulSoup(page.content, 'html.parser')

  resultados = soup.find('div', class_ = 'jsx-3672521041')
  opiniones = resultados.find_all('li', class_='jsx-571610088 post ')
  
  vector_notas = []
  vector_comentarios = []

  for opinion in opiniones:
      nota = opinion.find('div', class_='jsx-571610088 leftCalificacion').find('span', class_ = 'jsx-571610088 numeroStats')
      comentario = opinion.find('div', class_ = 'jsx-571610088 calificacion').find('div', class_ = 'jsx-571610088 lineBreak')

      vector_notas.append(nota.text)
      vector_comentarios.append(comentario.text)

  db = pd.DataFrame({'Nota': vector_notas, 'Comentario': vector_comentarios})
  frames.append(db)
  


Al final se organizan los dos datos en dos columnas distintas de un DataFrame.

In [0]:
data = pd.concat(frames)
data.to_csv('losestudiantes.csv', index = False)

pd.read_csv('losestudiantes.csv')
dataframe = pd.read_csv('losestudiantes.csv')
dataframe

,Nota,Comentario
0,4.0,"Explica lo más ameno que puede, hace las clase..."
1,4.5,El mejor profesor que he tenido hasta el momen...
2,4.5,Es excelente. La clase es bastante entretenida...
3,4.7,"Excelente profesor, deja tareas semanales con ..."
4,5.0,"Es un profesor con una actitud muy chévere, es..."
...,...,...
691,3.9,"es una buena persona, pero realmente no pareci..."
692,4.3,"Es buena explicando, a veces es un poco aburri..."
693,4.3,Explica muy bien aunque a veces comete pequeño...
694,4.4,"Tome el curso de EDO con ella, cuando recién t..."


# Diseño experimental

A diferencia del ejemplo de clasificador de películas nuestro problema no es binario pues el rango de notas está comprendido entre 1.5 y 5, y la nota que se le puede dar a un profesor es un número decimal entre ambos valores. Para dar solución a este primer obstáculo, se asocia cada posible valor a un número natural entre 0 y el numero de calificaciones que hay en la base de datos (en total hay aproximadamente 37 posibles valores para la nota), es decir se particionan los datos de acuerdo al natural (la clase) que se asocia a la calificación dada por el estudiante, así que hacemos una extensión del problema binario simplemente considerando más posibles clases que separan los datos, el éxito de esta extensión depende de qué tan diferenciadas puedan estar tales clases y esto depende a su vez del tipo de palabras que se encuentran en los comentarios asociados a una clase.

El modelo es regresión logística de manera que habrá una separación de las clases por medio de hiperplanos, sin embargo, dada la naturaleza binaria de la regresión es necesario definir una manera de evaluar el rendimiento de nuestro modelo de forma que tenga en cuenta su capacidad de clasificar los datos en alguna de las clases, esto se hace por medio del cociente entre el valor de la predicción y el natural que define la clase, el objetivo es que este valor sea un número muy cercano a 1. 

# Diseño metodológico



Aunque no es obligatorio, para probar el modelo podemos usar un comentario ficticio, este se almacena en una nueva línea del dataFrame creado arriba.

In [0]:
#se piden al usuario que de comentarios sobre un profesor y que lo calique
vector_comentarios2=[]
vector_notas2=[]
 
comentarios2=str(input('Ingresa los pros y los contras del profesor que vas a calificar: '))
nota2=float((input('Ingresa la calificacion que le asignas al profesor: ')))
vector_comentarios2.append(comentarios2)
vector_notas2.append(nota2)
datos_simulacion = pd.DataFrame({'Nota': vector_notas2, 'Comentario': vector_comentarios2})
datos_simulacion

Ingresa los pros y los contras del profesor que vas a calificar: el profesor es el mejor que he tenido en toda mi vida explica como los dioses pero es muy exigente
Ingresa la calificacion que le asignas al profesor: 4.5


,Nota,Comentario
0,4.5,el profesor es el mejor que he tenido en toda ...


In [0]:
#se anexa la calificasion y el comentario del usuario a la base de datos 
#(OJO, POR ERROR DESCONOCIDO HAY QUE ESCRIBIR DIRECTAMENTE LA NOTA INGRESADA POR EL USUARIO Y COPIAR Y PEGAR EL COMENTARIO)
a=int(len(dataframe))

dataframe.loc[a]=[4.5, ' el profesor es el mejor que he tenido en toda mi vida explica como los dioses pero es muy exigente']
dataframe

,Nota,Comentario
0,4.0,"Explica lo más ameno que puede, hace las clase..."
1,4.5,El mejor profesor que he tenido hasta el momen...
2,4.5,Es excelente. La clase es bastante entretenida...
3,4.7,"Excelente profesor, deja tareas semanales con ..."
4,5.0,"Es un profesor con una actitud muy chévere, es..."
...,...,...
693,4.3,Explica muy bien aunque a veces comete pequeño...
694,4.4,"Tome el curso de EDO con ella, cuando recién t..."
695,5.0,Es muy paciente y explica muy bien los temas. ...
696,4.5,el profesor es el mejor que he tenido en toda...


A continuación se muestra la cantidad de elementos que se agruparán en cada clase, como se mencionó antes la idea es poder clasificar los datos a partir de esta partición.

In [0]:
clase1=dataframe.groupby('Nota').size()
print(clase1)
len(clase1)

Nota
1.0      7
1.5     83
1.6      4
1.7      1
1.8      2
1.9      4
2.0     32
2.1      7
2.2      2
2.3      2
2.4      1
2.5     14
2.6      2
2.7      3
2.8      4
2.9      5
3.0     37
3.1      3
3.2      2
3.3      6
3.4      4
3.5     35
3.6      3
3.7      8
3.8      4
3.9      8
4.0     89
4.1      8
4.2      9
4.3     13
4.4     13
4.5     60
4.6     20
4.7     21
4.8     12
4.9      8
5.0    162
dtype: int64


37

In [0]:
#se le asigna un entero a cada nota
clase = [i for i in range(len(clase1)+1)]  # lista de clases equivalentes a las notas en un rango de 1.5 a 5.0
nota = [1.0]                               # lista de posibles notas
n = 1.5
for i in range (len(clase1)+1):
  nota.append(round(n, 1))
  n+=0.1

puntajeAsociado = []
for n in dataframe['Nota']:
  puntajeAsociado.append(nota.index(n))    # lista de los puntajes asociados a cada nota

df = pd.DataFrame({'Nota': dataframe['Nota'], 'clase': puntajeAsociado})
df

,Nota,clase
0,4.0,26
1,4.5,31
2,4.5,31
3,4.7,33
4,5.0,36
...,...,...
693,4.3,29
694,4.4,30
695,5.0,36
696,4.5,31


Para simplificar la ejecución del código y hacer más eficiente el modelo, hacemos una limpieza de los datos en el sentido de omitir información poco relevante y homogeneizar su estructura, para lograrlo se definen dos funciones: la primera se encarga de reemplazar mayúsculas por minúsculas, borrar signos de puntuación, corchetes, y palabras que contienen números entre sus letras, con la segunda nos deshacemos de las comillas. 

In [0]:
#se limpian los comentarios
import re
import string
 
def clean_text_round1(text):
    # pasa el texto a minúculas, elimina: paréntesis, signos de puntuación y números
    text = text.lower()
    text = re.sub('\[.*?¿\]\%', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
 
round1 = lambda x: clean_text_round1(x)
 
data_clean = pd.DataFrame(dataframe.Comentario.apply(round1))

def clean_text_round2(text):
    # se deshace de signos de puntuación adicionales y otros que no fueron eliminados en el primer filtro
    text = re.sub('[‘’“”…«»]', '', text)
    text = re.sub('\n', ' ', text)
    return text
 
round2 = lambda x: clean_text_round2(x)
 
data_clean = pd.DataFrame(data_clean.Comentario.apply(round2))

dataframe.to_pickle("corpus.pkl")
data_clean

,Comentario
0,explica lo más ameno que puede hace las clase...
1,el mejor profesor que he tenido hasta el momen...
2,es excelente la clase es bastante entretenida...
3,excelente profesor deja tareas semanales con ...
4,es un profesor con una actitud muy chévere es...
...,...
693,explica muy bien aunque a veces comete pequeño...
694,tome el curso de edo con ella cuando recién t...
695,es muy paciente y explica muy bien los temas ...
696,el profesor es el mejor que he tenido en toda...


Con el texto depurado ahora se construye el diccionario que caracterizará cada comentario, con la librería CountVectorizer construimos un vector de ceros, si el comentario no contiene la palabra del diccionario, y unos, si la contiene. Al final obtenemos una matriz cuyas filas corresponden a los comentarios dejados por los estudiantes y las columnas son cada palabra del diccionario construido.

In [0]:
#se eliminan las palabras no relevantes en base al archivo spanish.txt y se hace un conteo de palabras por cada comentario
from sklearn.feature_extraction.text import CountVectorizer
import pickle
 
with open('spanish.txt') as f:
    lines = f.read().splitlines()
 
cv = CountVectorizer(stop_words=lines)
data_cv = cv.fit_transform(data_clean.Comentario)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
 
data_dtm.to_pickle("dtm.pkl")

# El módulo pickle implementa protocolos binarios para serializar y deserializar 
# (convierte en una secuencia de bytes), una estructura de objetos Python
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))
 
data_dtm

,abajo,abarca,abarcar,abiertas,abierto,abjito,abordar,abra,abre,absoluta,absolutamente,absoluto,abstracta,abstractocons,abstractos,absurdament,absurdamente,abuelo,aburre,aburrida,aburridas,aburrido,aburridor,aburridorapros,aburridoras,abusivo,acabar,acabe,acabo,académica,académico,académicos,acaparar,acelera,acelerado,acento,acepta,aceptable,acepte,acerca,...,voy,voz,vpros,vueltas,vuelve,vuelven,ví,vídeos,vínculo,web,wikipedia,xd,yle,youtube,zalamea,zambrano,zapatos,álgebra,área,áreacons,élcons,élpros,énfasis,época,éstos,ética,éxito,íntegra,óptimo,última,últimas,último,últimos,única,únicamente,únicas,único,únicos,útil,útiles
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
694,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
695,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
696,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#se eliminan las filas correspondientes a los datos ingresados por el usuario
data_dtm4=data_dtm.drop(a,axis=0)
X = np.array(data_dtm4)
df4 = df.drop(a,axis=0)
y = np.array(df4['clase'])

El entrenamiento del modelo lo hacemos por medio de regresión logística, tomamos el 80% de los datos para entrenar y con el restante hacemos la evaluación.

In [0]:
#se entrena el modelo
model = linear_model.LogisticRegression()
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, y, test_size=validation_size, random_state=seed)
model.fit(X_train,Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Resultados

Si se evalúa el desempeño del modelo se encontrará una tasa de precisión insatisfactoria, esto se da por la incompatibilidad entre el caracter binario de la regresión logística y la tarea de clasificación múltiple. No obstante, al agregar los resultados de las predicciones y dividirlo por la suma de los valores de las notas de prueba se obtiene un valor cercano a 0.85, lo que indica que si bien la precisión es baja parece haber un sesgo no muy alto.

In [0]:
#score de los datos de validacion
predictions = model.predict(X_validation)
print(accuracy_score(Y_validation, predictions))

0.32142857142857145


In [0]:
#como el score anterior se calcula en base a un clasificador binario. A continuacion se da un indicador 
#más apropiado para este proyecto. Entre más sercano a 1 mejor predice. Si es mayor que 1 el modelo predijo en
#en exceso de lo contrario por defecto.

u = sum(predictions)
o = sum(Y_validation)
print(o/u)

0.8528114663726571


Finalmente, se hace una prueba adicional con la calificación fiticia que se ingresó al principio, al calcular el cociente entre predicciones y valores de prueba se nota una ligera mejora pues dicho valor se aproxima un poco más a 1.

In [0]:
#se agrega nuevamente los datos ingresados por el usuario y se da el score de toda la base de datos
X = np.array(data_dtm)
y = np.array(df['clase'])
predictions = model.predict(X)
print(accuracy_score(y, predictions))
u=sum(predictions)
o=sum(y)
print(o/u)

0.8567335243553008
0.9678597079243476


In [0]:
#se mustra los resultados de la prediccion de los datos ingresados por el usuario
h=predictions[-1]

nota=df[df['clase']==h]['Nota']
print('la nota que ingreso fue: ',nota2)
print('la nota que se predijo fue el último de la siguiente lista: ')
print(nota, h)

la nota que ingreso fue:  4.5
la nota que se predijo fue el último de la siguiente lista: 
4      5.0
5      5.0
10     5.0
11     5.0
15     5.0
      ... 
657    5.0
661    5.0
679    5.0
680    5.0
695    5.0
Name: Nota, Length: 162, dtype: float64 36


# Conclusiones

- En situaciones como la del proyecto, el scraping es una técnica apropiada para la extracción de datos de forma ordenada y eficiente. La librería de Python BeautifulSoup es una herramienta bastante útil para implementarlo.

- Es fundamental entender la macrostructura señalada por los datos para proponer un modelo estadístico lo más adecuado posible.

- Vale la pena intentar un enfoque estadístico alternativo para la solución del problema trabajado en el proyecto, pese a que se intentó extender la regresión logística para la clasificación multiclase quizás haya otra forma más adecuada para hacerlo.

- A pesar de que el modelo de regresión logistica es un clasificador binario, en este proyecto predice bastante bien.

- A pesar de que el modelo funciona de manera adecuada, sería pertinente implementar algo adicional que permita el mejoramiento de este. Para ello, se sugiere la retroalimentación de los comentarios que se van ingresando, de manera que se puedan ampliar el diccionario de palabras y los datos de entrenamiento. Para esto se pueden agrupar las acciones implementadas en funciones y crear una nueva que tome los datos que se ingresan y los agregue a la base de datos para entrenar el modelo nuevamente.
```
# ejemplo de función posible a implementar
def retroalimentarModelo(nota_, comentario_):
  a = int(len(dataframe))
  dataframe.loc[a]=[nota_, comentario_] #agrega la información nueva a la base de datos
  modelTrain(makeList(cleanData(dataframe)), asociarPuntaje(dataframe)) #se entrena nuevamente el modelo

